1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)
2. Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)
3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты. 
4. (опционально) Провести подбор гиперпараметров лучшей модели по итогам 2-3
5. (опционально) Еще раз провести оценку экономической эффективности

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
from sklearn.preprocessing import StandardScaler

from catboost import CatBoostClassifier

import itertools

import matplotlib.pyplot as plt

Загрузим датасет, сделаем разбиение на тренировочную и тестовую выборки

In [2]:
df = pd.read_csv("churn_data.csv")
df.drop(columns='CustomerId', inplace=True)
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=42)

In [3]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

Определим колонки, которые отнесем к категориальным и к численным. Для категориальных применим OHE, а численные стандартизируем.

In [4]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [5]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col)),
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('stand', StandardScaler()),
            ])
    final_transformers.append((cont_col, cont_transformer))

Объединим всё это в единый пайплайн

In [6]:
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

Первой применим модель с градиентным бустингом

In [8]:
pipeline = Pipeline([
    ('features',feats),
    ('classifier', GradientBoostingClassifier(random_state = 42)),
])

In [9]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [10]:
preds_gb = pipeline.predict_proba(X_test)[:, 1]

In [11]:
precision_gb, recall_gb, thresholds_gb = precision_recall_curve(y_test, preds_gb)

fscore_gb = (2 * precision_gb * recall_gb) / (precision_gb + recall_gb)

ix_gb = np.argmax(fscore_gb)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds_gb[ix_gb], 
                                                                        fscore_gb[ix_gb],
                                                                        precision_gb[ix_gb],
                                                                        recall_gb[ix_gb]))

Best Threshold=0.271906, F-Score=0.642, Precision=0.602, Recall=0.686


Второй попробуем логистическую регресиию

In [12]:
pipeline_lr = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42)),
])

In [13]:
pipeline_lr.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [14]:
preds_lr = pipeline_lr.predict_proba(X_test)[:, 1]

In [15]:
precision_lr, recall_lr, thresholds_lr = precision_recall_curve(y_test, preds_lr)

fscore_lr = (2 * precision_lr * recall_lr) / (precision_lr + recall_lr + 1e-10)

ix_lr = np.argmax(fscore_lr)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds_lr[ix_lr], 
                                                                        fscore_lr[ix_lr],
                                                                        precision_lr[ix_lr],
                                                                        recall_lr[ix_lr]))

Best Threshold=0.274065, F-Score=0.503, Precision=0.443, Recall=0.581


И CatBoost

In [16]:
pipeline_cb = Pipeline([
    ('features',feats),
    ('classifier', CatBoostClassifier(random_state = 42)),
    
])

In [17]:
pipeline_cb.fit(X_train, y_train);

Learning rate set to 0.024355
0:	learn: 0.6726918	total: 182ms	remaining: 3m 2s
1:	learn: 0.6545033	total: 210ms	remaining: 1m 44s
2:	learn: 0.6400397	total: 241ms	remaining: 1m 19s
3:	learn: 0.6231921	total: 274ms	remaining: 1m 8s
4:	learn: 0.6071283	total: 302ms	remaining: 1m
5:	learn: 0.5952178	total: 322ms	remaining: 53.3s
6:	learn: 0.5810797	total: 350ms	remaining: 49.6s
7:	learn: 0.5690754	total: 375ms	remaining: 46.5s
8:	learn: 0.5570439	total: 404ms	remaining: 44.5s
9:	learn: 0.5444553	total: 440ms	remaining: 43.6s
10:	learn: 0.5331415	total: 478ms	remaining: 42.9s
11:	learn: 0.5221842	total: 510ms	remaining: 42s
12:	learn: 0.5132382	total: 538ms	remaining: 40.9s
13:	learn: 0.5054701	total: 568ms	remaining: 40s
14:	learn: 0.4982012	total: 617ms	remaining: 40.5s
15:	learn: 0.4901325	total: 649ms	remaining: 39.9s
16:	learn: 0.4838581	total: 687ms	remaining: 39.7s
17:	learn: 0.4777220	total: 778ms	remaining: 42.5s
18:	learn: 0.4707221	total: 911ms	remaining: 47.1s
19:	learn: 0.463

164:	learn: 0.3191530	total: 9.83s	remaining: 49.8s
165:	learn: 0.3189485	total: 9.89s	remaining: 49.7s
166:	learn: 0.3187577	total: 9.95s	remaining: 49.6s
167:	learn: 0.3185637	total: 9.98s	remaining: 49.4s
168:	learn: 0.3183840	total: 10.1s	remaining: 49.4s
169:	learn: 0.3181938	total: 10.1s	remaining: 49.5s
170:	learn: 0.3180945	total: 10.2s	remaining: 49.5s
171:	learn: 0.3178700	total: 10.3s	remaining: 49.4s
172:	learn: 0.3177650	total: 10.3s	remaining: 49.5s
173:	learn: 0.3176553	total: 10.4s	remaining: 49.5s
174:	learn: 0.3175426	total: 10.5s	remaining: 49.3s
175:	learn: 0.3174107	total: 10.5s	remaining: 49.3s
176:	learn: 0.3171519	total: 10.6s	remaining: 49.1s
177:	learn: 0.3170200	total: 10.6s	remaining: 49.1s
178:	learn: 0.3168925	total: 10.7s	remaining: 48.9s
179:	learn: 0.3167568	total: 10.8s	remaining: 49.2s
180:	learn: 0.3166196	total: 10.8s	remaining: 49.1s
181:	learn: 0.3164862	total: 10.9s	remaining: 49s
182:	learn: 0.3162975	total: 10.9s	remaining: 48.9s
183:	learn: 0.

327:	learn: 0.2975483	total: 16.2s	remaining: 33.1s
328:	learn: 0.2974622	total: 16.2s	remaining: 33s
329:	learn: 0.2973733	total: 16.2s	remaining: 32.9s
330:	learn: 0.2973123	total: 16.3s	remaining: 32.8s
331:	learn: 0.2971709	total: 16.3s	remaining: 32.7s
332:	learn: 0.2970771	total: 16.3s	remaining: 32.7s
333:	learn: 0.2969889	total: 16.3s	remaining: 32.6s
334:	learn: 0.2968857	total: 16.4s	remaining: 32.5s
335:	learn: 0.2966922	total: 16.4s	remaining: 32.4s
336:	learn: 0.2966000	total: 16.5s	remaining: 32.5s
337:	learn: 0.2964920	total: 16.6s	remaining: 32.5s
338:	learn: 0.2963160	total: 16.6s	remaining: 32.5s
339:	learn: 0.2961755	total: 16.7s	remaining: 32.4s
340:	learn: 0.2960598	total: 16.7s	remaining: 32.4s
341:	learn: 0.2959440	total: 16.8s	remaining: 32.3s
342:	learn: 0.2958350	total: 16.8s	remaining: 32.2s
343:	learn: 0.2955770	total: 16.9s	remaining: 32.2s
344:	learn: 0.2954718	total: 16.9s	remaining: 32.1s
345:	learn: 0.2953064	total: 16.9s	remaining: 32s
346:	learn: 0.29

486:	learn: 0.2788144	total: 21.3s	remaining: 22.4s
487:	learn: 0.2787209	total: 21.3s	remaining: 22.4s
488:	learn: 0.2786047	total: 21.3s	remaining: 22.3s
489:	learn: 0.2785574	total: 21.4s	remaining: 22.2s
490:	learn: 0.2784301	total: 21.4s	remaining: 22.2s
491:	learn: 0.2783443	total: 21.4s	remaining: 22.1s
492:	learn: 0.2782866	total: 21.5s	remaining: 22.1s
493:	learn: 0.2782095	total: 21.5s	remaining: 22.1s
494:	learn: 0.2781525	total: 21.6s	remaining: 22s
495:	learn: 0.2780133	total: 21.6s	remaining: 21.9s
496:	learn: 0.2779407	total: 21.6s	remaining: 21.9s
497:	learn: 0.2778148	total: 21.6s	remaining: 21.8s
498:	learn: 0.2776965	total: 21.7s	remaining: 21.7s
499:	learn: 0.2775610	total: 21.7s	remaining: 21.7s
500:	learn: 0.2774529	total: 21.7s	remaining: 21.6s
501:	learn: 0.2773670	total: 21.7s	remaining: 21.5s
502:	learn: 0.2772507	total: 21.7s	remaining: 21.5s
503:	learn: 0.2771178	total: 21.8s	remaining: 21.4s
504:	learn: 0.2769602	total: 21.8s	remaining: 21.3s
505:	learn: 0.

650:	learn: 0.2618163	total: 25.5s	remaining: 13.7s
651:	learn: 0.2617135	total: 25.6s	remaining: 13.7s
652:	learn: 0.2616442	total: 25.6s	remaining: 13.6s
653:	learn: 0.2615660	total: 25.7s	remaining: 13.6s
654:	learn: 0.2614855	total: 25.7s	remaining: 13.5s
655:	learn: 0.2613792	total: 25.7s	remaining: 13.5s
656:	learn: 0.2612799	total: 25.8s	remaining: 13.5s
657:	learn: 0.2611644	total: 25.8s	remaining: 13.4s
658:	learn: 0.2610663	total: 25.8s	remaining: 13.4s
659:	learn: 0.2609773	total: 25.8s	remaining: 13.3s
660:	learn: 0.2608647	total: 25.9s	remaining: 13.3s
661:	learn: 0.2607805	total: 25.9s	remaining: 13.2s
662:	learn: 0.2607120	total: 26s	remaining: 13.2s
663:	learn: 0.2606052	total: 26.1s	remaining: 13.2s
664:	learn: 0.2604986	total: 26.1s	remaining: 13.2s
665:	learn: 0.2604138	total: 26.2s	remaining: 13.1s
666:	learn: 0.2603558	total: 26.2s	remaining: 13.1s
667:	learn: 0.2602671	total: 26.3s	remaining: 13.1s
668:	learn: 0.2602103	total: 26.3s	remaining: 13s
669:	learn: 0.26

817:	learn: 0.2466593	total: 30.1s	remaining: 6.69s
818:	learn: 0.2466116	total: 30.1s	remaining: 6.65s
819:	learn: 0.2464887	total: 30.1s	remaining: 6.61s
820:	learn: 0.2464097	total: 30.1s	remaining: 6.57s
821:	learn: 0.2463358	total: 30.1s	remaining: 6.52s
822:	learn: 0.2462850	total: 30.1s	remaining: 6.48s
823:	learn: 0.2461594	total: 30.2s	remaining: 6.44s
824:	learn: 0.2460527	total: 30.2s	remaining: 6.4s
825:	learn: 0.2459237	total: 30.2s	remaining: 6.36s
826:	learn: 0.2458565	total: 30.2s	remaining: 6.32s
827:	learn: 0.2457789	total: 30.2s	remaining: 6.28s
828:	learn: 0.2457729	total: 30.3s	remaining: 6.24s
829:	learn: 0.2456969	total: 30.3s	remaining: 6.2s
830:	learn: 0.2455798	total: 30.3s	remaining: 6.16s
831:	learn: 0.2455276	total: 30.3s	remaining: 6.12s
832:	learn: 0.2454001	total: 30.4s	remaining: 6.09s
833:	learn: 0.2453003	total: 30.4s	remaining: 6.05s
834:	learn: 0.2452460	total: 30.4s	remaining: 6.01s
835:	learn: 0.2451832	total: 30.5s	remaining: 5.98s
836:	learn: 0.

981:	learn: 0.2326190	total: 34.2s	remaining: 627ms
982:	learn: 0.2324829	total: 34.2s	remaining: 592ms
983:	learn: 0.2324023	total: 34.3s	remaining: 557ms
984:	learn: 0.2323312	total: 34.3s	remaining: 522ms
985:	learn: 0.2322456	total: 34.3s	remaining: 487ms
986:	learn: 0.2321915	total: 34.3s	remaining: 452ms
987:	learn: 0.2321514	total: 34.4s	remaining: 418ms
988:	learn: 0.2320720	total: 34.4s	remaining: 383ms
989:	learn: 0.2319801	total: 34.5s	remaining: 348ms
990:	learn: 0.2319109	total: 34.5s	remaining: 313ms
991:	learn: 0.2318190	total: 34.5s	remaining: 278ms
992:	learn: 0.2317645	total: 34.5s	remaining: 243ms
993:	learn: 0.2316352	total: 34.5s	remaining: 209ms
994:	learn: 0.2315390	total: 34.6s	remaining: 174ms
995:	learn: 0.2315335	total: 34.6s	remaining: 139ms
996:	learn: 0.2314603	total: 34.6s	remaining: 104ms
997:	learn: 0.2313569	total: 34.6s	remaining: 69.4ms
998:	learn: 0.2312453	total: 34.6s	remaining: 34.7ms
999:	learn: 0.2311588	total: 34.7s	remaining: 0us


In [18]:
preds_cb = pipeline_cb.predict_proba(X_test)[:, 1]

In [19]:
precision_cb, recall_cb, thresholds_cb = precision_recall_curve(y_test, preds_cb)

fscore_cb = (2 * precision_cb * recall_cb) / (precision_cb + recall_cb)

ix_cb = np.argmax(fscore_cb)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds_cb[ix_cb], 
                                                                        fscore_cb[ix_cb],
                                                                        precision_cb[ix_cb],
                                                                        recall_cb[ix_cb]))

Best Threshold=0.296142, F-Score=0.646, Precision=0.613, Recall=0.682


In [20]:
result_table = pd.DataFrame({
    'Gradient Boosting': pd.Series({
        'Thresholds': thresholds_gb[ix_gb], 
        'FScore': fscore_gb[ix_gb], 
        'Precision': precision_gb[ix_gb], 
        'Recall': recall_gb[ix_gb]
    }),
    'Linear Regression': pd.Series({
        'Thresholds': thresholds_lr[ix_lr], 
        'FScore': fscore_lr[ix_lr], 
        'Precision': precision_lr[ix_lr], 
        'Recall': recall_lr[ix_lr]
    }),
    'Cat Boosting': pd.Series({
        'Thresholds': thresholds_cb[ix_cb], 
        'FScore': fscore_cb[ix_cb], 
        'Precision': precision_cb[ix_cb], 
        'Recall': recall_cb[ix_cb]
    }),
})

result_table

,Gradient Boosting,Linear Regression,Cat Boosting
Thresholds,0.271906,0.274065,0.296142
FScore,0.641580,0.502609,0.645714
Precision,0.602473,0.442573,0.613020
Recall,0.686117,0.581489,0.682093


Как видим, категориальный бустинг работает лучше всего, хотя градиентный примерно также (на значение F-score не сильно влияет дисбаланс классов, поэтому использую ее для оценки).

In [21]:
conf_matrix_cb = confusion_matrix(y_test, preds_cb > thresholds_cb[ix_cb])

Посчитаем выручку с применением модели Cat Boost

In [22]:
profit = 2*conf_matrix_cb[1, 1]-(conf_matrix_cb[0, 1]+conf_matrix_cb[1, 1])
print(f'{profit}$')

124$
